# データサイエンス100本ノック（構造化データ加工編） - Python

## はじめに
- 初めに以下のセルを実行してください
- 必要なライブラリのインポートとデータベース（PostgreSQL）からのデータ読み込みを行います
- pandas等、利用が想定されるライブラリは以下セルでインポートしています
- その他利用したいライブラリがあれば適宜インストールしてください（"!pip install ライブラリ名"でインストールも可能）
- 処理は複数回に分けても構いません
- 名前、住所等はダミーデータであり、実在するものではありません

In [3]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta
from dateutil.relativedelta import relativedelta
import math
import psycopg2
from sqlalchemy import create_engine
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler # conda install -c conda-forge imbalanced-learn

df_customer = pd.read_csv("./data/customer.csv")
df_category = pd.read_csv("./data/category.csv")
df_product = pd.read_csv("./data/product.csv")
df_receipt = pd.read_csv("./data/receipt.csv")
df_store = pd.read_csv("./data/store.csv")
df_geocode = pd.read_csv("./data/geocode.csv")

/Users/miwa/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
/Users/miwa/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# 演習問題

---
> P-071: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からの経過月数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1ヶ月未満は切り捨てること。

In [4]:
df_tmp = pd.merge(df_receipt[['customer_id', 'sales_ymd']], df_customer[['customer_id', 'application_date']], how='inner', on='customer_id')
df_tmp = df_tmp.drop_duplicates()

df_tmp['sales_ymd'] = pd.to_datetime(df_tmp['sales_ymd'].astype('str'))
df_tmp['application_date'] = pd.to_datetime(df_tmp['application_date'].astype('str'))

# relativedelta(x['sales_ymd'], x['application_date']).monthsで月差分が出せるはず…？
# https://qiita.com/recuraki/items/62f71794e235d3592090

# 下記でsyntaxEorrorで解けなかった

# def calc_month_diff(x):
#     diff = relativedelta(x['sales_ymd'], x['application_date']).months, axis=1)
#     return diff.years*12+diff.months

# df_tmp_month = df_tmp.apply(calc_month_diff, axis=1).reset_index()
# df_tmp_month.columns = ['index', 'elapsed_month']
# df_tmp['elapsed_month'] = df_tmp_month['elapsed_month']
# df_tmp.head(10)

# 答え
df_tmp['elapsed_date'] = df_tmp[['sales_ymd', 'application_date']].apply(lambda x:
                                                                         relativedelta(x[0],x[1]).years*12 + relativedelta(x[0],x[1]).months,axis=1)
df_tmp.sort_values('customer_id').head(10)


,customer_id,sales_ymd,application_date,elapsed_date
60376,CS001113000004,2019-03-08,2015-11-05,40
20158,CS001114000005,2019-07-31,2016-04-12,39
20156,CS001114000005,2018-05-03,2016-04-12,24
29140,CS001115000010,2019-04-05,2015-04-17,47
29141,CS001115000010,2018-07-01,2015-04-17,38
29142,CS001115000010,2017-12-28,2015-04-17,32
6526,CS001205000004,2019-06-25,2016-06-15,36
6521,CS001205000004,2019-03-12,2016-06-15,32
6518,CS001205000004,2018-08-21,2016-06-15,26
6520,CS001205000004,2017-09-14,2016-06-15,14


---
> P-072: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からの経過年数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い。（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1年未満は切り捨てること。

In [41]:
# 自分答えが正解

df_tmp = pd.merge(df_receipt[['customer_id', 'sales_ymd']], df_customer[['customer_id', 'application_date']], how='inner', on='customer_id')
df_tmp = df_tmp.drop_duplicates()

df_tmp['sales_ymd'] = pd.to_datetime(df_tmp['sales_ymd'].astype('str'))
df_tmp['application_date'] = pd.to_datetime(df_tmp['application_date'].astype('str'))

# relativedelta(x['sales_ymd'], x['application_date']).monthsで月差分が出せるはず…？
# https://qiita.com/recuraki/items/62f71794e235d3592090
df_tmp_year = df_tmp.apply(lambda x: relativedelta(x['sales_ymd'], x['application_date']).years, axis=1).reset_index()
df_tmp_year.columns = ['index', 'elapsed_year']
df_tmp['elapsed_year'] = df_tmp_year['elapsed_year']
df_tmp.head(10)

#解答
# df_tmp = pd.merge(df_receipt[['customer_id', 'sales_ymd']], df_customer[['customer_id', 'application_date']],
#                  how='inner', on='customer_id')

# df_tmp['sales_ymd'] = pd.to_datetime(df_tmp['sales_ymd'].astype('str'))
# df_tmp['application_date'] = pd.to_datetime(df_tmp['application_date'].astype('str')) # 修正

# df_tmp['elapsed_date'] = df_tmp[['sales_ymd', 'application_date']].apply(lambda x: 
#                                                                          relativedelta(x[0], x[1]).years, axis=1)
# df_tmp.head(10)

,customer_id,sales_ymd,application_date,elapsed_year
0,CS006214000001,2018-11-03,2015-02-01,3.0
1,CS006214000001,2017-05-09,2015-02-01,2.0
2,CS006214000001,2017-06-08,2015-02-01,2.0
4,CS006214000001,2018-10-28,2015-02-01,4.0
7,CS006214000001,2019-09-08,2015-02-01,3.0
8,CS006214000001,2018-01-31,2015-02-01,4.0
9,CS006214000001,2017-07-05,2015-02-01,4.0
10,CS006214000001,2018-11-10,2015-02-01,4.0
12,CS006214000001,2019-04-10,2015-02-01,2.0
15,CS006214000001,2019-06-01,2015-02-01,2.0


---
> P-073: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からのエポック秒による経過時間を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。なお、時間情報は保有していないため各日付は0時0分0秒を表すものとする。

In [5]:
# 途中で分からなくなった

# df_tmp = pd.merge(df_receipt[['customer_id', 'sales_ymd']], df_customer[['customer_id', 'application_date']], how='inner', on='customer_id')
# df_tmp = df_tmp.drop_duplicates()

# df_tmp['sales_ymd'] = pd.to_datetime(df_tmp['sales_ymd'].astype('str'))
# df_tmp['application_date'] = pd.to_datetime(df_tmp['application_date'].astype('int'))

# df_tmp['elapsed_epoc'] = 
# df_tmp.head(10)

# 答え

df_tmp = pd.merge(df_receipt[['customer_id', 'sales_ymd']], df_customer[['customer_id', 'application_date']],
                 how='inner', on='customer_id')

df_tmp = df_tmp.drop_duplicates()

df_tmp['sales_ymd'] = pd.to_datetime(df_tmp['sales_ymd'].astype('str'))
df_tmp['application_date'] = pd.to_datetime(df_tmp['application_date'].astype('str')) 

df_tmp['elapsed_date'] = (df_tmp['sales_ymd'].astype(np.int64) / 10**9)-(df_tmp['application_date'].astype(np.int64) / 10**9)
df_tmp.head(10)

,customer_id,sales_ymd,application_date,elapsed_date
0,CS006214000001,2018-11-03,2015-02-01,118454400.0
1,CS006214000001,2017-05-09,2015-02-01,71539200.0
2,CS006214000001,2017-06-08,2015-02-01,74131200.0
4,CS006214000001,2018-10-28,2015-02-01,117936000.0
7,CS006214000001,2019-09-08,2015-02-01,145152000.0
8,CS006214000001,2018-01-31,2015-02-01,94608000.0
9,CS006214000001,2017-07-05,2015-02-01,76464000.0
10,CS006214000001,2018-11-10,2015-02-01,119059200.0
12,CS006214000001,2019-04-10,2015-02-01,132105600.0
15,CS006214000001,2019-06-01,2015-02-01,136598400.0


---
> P-074: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、当該週の月曜日からの経過日数を計算し、売上日、当該週の月曜日付とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値でデータを保持している点に注意）。

In [6]:
# 自分の解答

# def calc_monday(x):
#     week_num = x.weekday()
#     x - pd.tseries.offsets.Day(week_num)

# df_tmp = df_receipt.copy()
# df_tmp['sales_ymd'] = pd.to_datetime(df_tmp['sales_ymd'].astype('str'))
# df_tmp['sales_ymd_offset'] = df_tmp['sales_ymd'].apply(calc_monday)
# df_tmp.head(10)

#答え
df_tmp = df_receipt[['customer_id', 'sales_ymd']]
df_tmp = df_tmp.drop_duplicates()
df_tmp['sales_ymd'] = pd.to_datetime(df_tmp['sales_ymd'].astype('str'))
df_tmp['monday'] = df_tmp['sales_ymd'].apply(lambda x: x - relativedelta(days=x.weekday()))
df_tmp['elapsed_weekday'] = df_tmp['sales_ymd'] - df_tmp['monday']
df_tmp.head(10)

,customer_id,sales_ymd,monday,elapsed_weekday
0,CS006214000001,2018-11-03,2018-10-29,5 days
1,CS008415000097,2018-11-18,2018-11-12,6 days
2,CS028414000014,2017-07-12,2017-07-10,2 days
3,ZZ000000000000,2019-02-05,2019-02-04,1 days
4,CS025415000050,2018-08-21,2018-08-20,1 days
5,CS003515000195,2019-06-05,2019-06-03,2 days
6,CS024514000042,2018-12-05,2018-12-03,2 days
7,CS040415000178,2019-09-22,2019-09-16,6 days
8,ZZ000000000000,2017-05-04,2017-05-01,3 days
9,CS027514000015,2019-10-10,2019-10-07,3 days


---
> P-075: 顧客データフレーム（df_customer）からランダムに1%のデータを抽出し、先頭から10件データを抽出せよ。

In [13]:
df_customer.sample(frac=0.01, replace=True, random_state=1).head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
235,CS016415000181,水田 奈々,1,女性,1977-07-11,41,184-0012,東京都小金井市中町**********,S13016,20151025,E-20101027-D
12172,CS003415000866,小市 沙知絵,1,女性,1975-01-26,44,201-0013,東京都狛江市元和泉**********,S13003,20170220,0-00000000-0
5192,CS009415000127,矢口 美紀,1,女性,1977-06-23,41,158-0097,東京都世田谷区用賀**********,S13009,20151207,A-20100916-D
17289,CS021514000073,熊沢 美佐子,1,女性,1962-02-23,57,259-1143,神奈川県伊勢原市下糟屋**********,S14021,20151105,F-20100721-F
10955,CS033401000014,六角 長利,0,男性,1973-08-12,45,245-0009,神奈川県横浜市泉区新橋町**********,S14033,20150406,0-00000000-0
7813,CS004314000100,小笠原 真帆,1,女性,1981-06-11,37,165-0027,東京都中野区野方**********,S13004,20160426,6-20101019-8
19279,CS038502000054,大山 裕司,0,男性,1962-12-07,56,134-0083,東京都江戸川区中葛西**********,S13038,20150914,0-00000000-0
21440,CS039315000088,秦 夏希,1,女性,1984-04-26,34,167-0051,東京都杉並区荻窪**********,S13039,20150614,5-20081025-6
144,CS020412000161,小宮 薫,1,女性,1974-05-21,44,174-0042,東京都板橋区東坂下**********,S13020,20150822,B-20081021-3
20609,CS001513000165,白川 由樹,1,女性,1959-11-14,59,144-0052,東京都大田区蒲田**********,S13001,20150711,0-00000000-0
